In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("liverpool-players-analysis").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

22/08/13 22:52:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
liverpool_players = spark.read.csv(path="../data/liverpool_players.csv", sep=",", header=True)

In [3]:
liverpool_players.show(n=10)

+------+--------------------+----------+-----------+-----------+
|    id|                name|  position|dateOfBirth|nationality|
+------+--------------------+----------+-----------+-----------+
|  1795|             Alisson|Goalkeeper| 1992-10-02|     Brazil|
|  8197|              Adrián|Goalkeeper| 1987-01-03|      Spain|
|102046|   Caoimhin Kelleher|Goalkeeper| 1998-11-23|    Ireland|
|  7383|     Kostas Tsimikas|   Defence| 1996-05-12|     Greece|
|  7720|   Sepp van den Berg|   Defence| 2001-12-20|Netherlands|
|  7861|        James Milner|   Defence| 1986-01-04|    England|
|  7862|           Joe Gomez|   Defence| 1997-05-23|    England|
|  7865|          Joël Matip|   Defence| 1991-08-08|   Cameroon|
|  7867|Trent Alexander-A...|   Defence| 1998-10-07|    England|
|  7868|    Andrew Robertson|   Defence| 1994-03-11|   Scotland|
+------+--------------------+----------+-----------+-----------+
only showing top 10 rows



In [4]:
top_scorers_pl = spark.read.csv(path="../data/top_scorers_pl.csv", sep=",", header=True)

In [5]:
top_scorers_pl.show(n=10)

+----+-----------------+-----------------+-----------------+-----------+--------------+--------+-----------+--------------------+
|  id|             name|        firstName|         lastName|dateOfBirth|   nationality|position|shirtNumber|         lastUpdated|
+----+-----------------+-----------------+-----------------+-----------+--------------+--------+-----------+--------------------+
|3754|    Mohamed Salah|          Mohamed|            Salah| 1992-06-15|         Egypt| Offence|       null|2022-04-24T17:48:30Z|
|3343|     Son Heungmin|         Heungmin|              Son| 1992-07-08|Korea Republic|Midfield|       null|2021-10-13T08:04:15Z|
|  44|Cristiano Ronaldo|Cristiano Ronaldo|dos Santos Aveiro| 1985-02-05|      Portugal| Offence|       null|2021-10-13T08:04:10Z|
|8004|       Harry Kane|            Harry|             null| 1993-07-28|       England| Offence|       null|2020-11-26T02:25:00Z|
|3626|       Sadio Mané|            Sadio|             null| 1992-04-10|       Senegal| Of

In [14]:
top_liverpool_scorers = liverpool_players.alias("lp").join(
    top_scorers_pl.alias("ts"),
    ['id'],
    "inner"
).select("lp.id", "ts.name", "ts.firstName", "ts.lastName", "ts.dateOfBirth", "ts.nationality", "ts.position")

In [15]:
top_liverpool_scorers.show()

+----+-------------+----------+--------+-----------+-----------+--------+
|  id|         name| firstName|lastName|dateOfBirth|nationality|position|
+----+-------------+----------+--------+-----------+-----------+--------+
|3754|Mohamed Salah|   Mohamed|   Salah| 1992-06-15|      Egypt| Offence|
|4092|   Diogo Jota|Diogo José|    null| 1996-12-04|   Portugal| Offence|
+----+-------------+----------+--------+-----------+-----------+--------+



In [17]:
top_liverpool_scorers.write.csv("../data/top_liverpool_scorers")